<a href="https://colab.research.google.com/github/RZx-Taishou/Facial-Pain-Recognition/blob/main/Facial-Pain-Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet152V2

In [ ]:
!git clone https://github.com/RZx-Taishou/Facial-Pain-Recognition.git

fatal: destination path 'Facial-Pain-Recognition' already exists and is not an empty directory.


In [ ]:
ORIGINAL_DIR = "/content/Facial-Pain-Recognition/Dataset"

In [ ]:
training_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=.2
)

validation_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=.2
)

train_generator = training_datagen.flow_from_directory(
    ORIGINAL_DIR,
    target_size=(150, 150),
    class_mode="binary",
    batch_size=64,
    subset="training"
)

validation_generator = validation_datagen.flow_from_directory(
    ORIGINAL_DIR,
    target_size=(150, 150),
    class_mode="binary",
    batch_size=32,
    subset="validation"
)

Found 718 images belonging to 2 classes.
Found 178 images belonging to 2 classes.


In [ ]:
effb7 = ResNet152V2(input_shape=(150, 150, 3), include_top=False, weights=None)

for layer in effb7.layers:
  layer.trainable = False

x = effb7.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
#x = tf.keras.layers.Dropout(.2)(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = Model(effb7.input, x)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 77, 77, 64)   0           ['conv1_conv[0][0]']       

In [ ]:
loss = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True,
    alpha=0.25,
    gamma=2.0,
    from_logits=False,
)

In [ ]:
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [ ]:
model.fit(train_generator, epochs=100, validation_data=validation_generator, verbose=2)

Epoch 1/100
12/12 - 78s - loss: 0.0689 - accuracy: 0.7270 - val_loss: 0.0668 - val_accuracy: 0.7303 - 78s/epoch - 6s/step
Epoch 2/100
12/12 - 57s - loss: 0.0670 - accuracy: 0.7270 - val_loss: 0.0667 - val_accuracy: 0.7303 - 57s/epoch - 5s/step
Epoch 3/100
12/12 - 57s - loss: 0.0670 - accuracy: 0.7270 - val_loss: 0.0667 - val_accuracy: 0.7303 - 57s/epoch - 5s/step
Epoch 4/100
12/12 - 55s - loss: 0.0670 - accuracy: 0.6908 - val_loss: 0.0667 - val_accuracy: 0.2697 - 55s/epoch - 5s/step
Epoch 5/100
12/12 - 56s - loss: 0.0669 - accuracy: 0.2730 - val_loss: 0.0667 - val_accuracy: 0.2697 - 56s/epoch - 5s/step
Epoch 6/100
12/12 - 55s - loss: 0.0669 - accuracy: 0.2730 - val_loss: 0.0666 - val_accuracy: 0.2697 - 55s/epoch - 5s/step
Epoch 7/100
12/12 - 54s - loss: 0.0669 - accuracy: 0.2730 - val_loss: 0.0666 - val_accuracy: 0.2697 - 54s/epoch - 5s/step
Epoch 8/100
12/12 - 55s - loss: 0.0669 - accuracy: 0.2730 - val_loss: 0.0794 - val_accuracy: 0.7303 - 55s/epoch - 5s/step
Epoch 9/100
12/12 - 54s 

In [ ]:
model.save("FPR.h5")

In [ ]:
from google.colab import files
files.download("FPR.h5")